In [1]:
from pyspark import SparkFiles
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pyspark'

In [0]:
# from pyspark import SparkFiles
# url = "https://proai-datasets.s3.eu-west-3.amazonaws.com/wikipedia.csv"

# sc.addFile(url)
# df = spark.read.csv("file://"+SparkFiles.get("wikipedia.csv"), header = True, quote='\"', escape='\"')
# df = df.drop("_c0")
# df = df.withColumnRenamed("categoria","category")
# df.printSchema()
# df.show()

In [0]:
# df.head()["documents"]

In [0]:


url = "https://proai-datasets.s3.eu-west-3.amazonaws.com/wikipedia.csv"

sc.addFile(url)
spark_df = spark.read.csv("file://"+SparkFiles.get("wikipedia.csv"), header = True, quote='\"', escape='\"')
spark_df = spark_df.drop("_c0")
spark_df = spark_df.withColumnRenamed("categoria","category")
spark_df.printSchema()
spark_df.show()

_ = spark.sql("DROP TABLE IF EXISTS wikipedia")
spark_df.write.saveAsTable("wikipedia")

In [0]:
%sql
-- check
SELECT * FROM wikipedia LIMIT 10

# EDA - Explorative data analysis
The EDA aims to understand wikipedia articles features, related to their category  

## Articles counts for each category

In [0]:
%sql
SELECT
  w.category,
  COUNT(*) AS count
FROM wikipedia w
GROUP BY w.category


In [0]:
result_set = _sqldf.collect()
categories = [row["category"] for row in result_set]
counts = [row["count"] for row in result_set]

plt.figure()
plt.bar(categories, counts)
plt.xlabel("Category")
plt.ylabel("Frequency")
plt.title("Category absolute frequency distribution")
plt.xticks(rotation=45)
plt.show()

#TODO improve the chart

The dataset is quite balanced (in terms of categories), even if `politics` category is a bit more represented than other ones.

## Average words count in each article
Articles are stored in `documents` column

documents -> string split [tokenizer] -> length of each split -> mean of these lengths

In [0]:
import string
import re
from pyspark.sql import Row

def remove_punctuation_and_numbers(row: Row) -> Row:
    """
    Compute a raw text cleaning, by removing punctuation, only digits words and replce multiple spaces with single ones.

    Words with only digits and punctuation are removed too, like doi numbers, example: 10.1080/13501780801913298
    """

    sentence: str = row["documents"]

    for c in string.punctuation:
        sentence = sentence.replace(c, " ")
    
    #remove on
    sentence = re.sub(r"\b\d+\b","", sentence)

    #remove multiple spaces
    sentence = re.sub(r" +"," ",sentence)

    

    #questo qui sotto è per le stopwords(dopo)
    #clean_sentence = " ".join([word for word in sentence.split() if word not in string.punctuation])

    return Row(documents = row["documents"], documents_clean=sentence)


In [0]:
#check 
example = "this is a doi number: 10.1080/13501780801913298 bla bla another number: 1256 eee"
remove_punctuation_and_numbers(Row(documents = example))
sentence = "this is a doi number: 10.1080/13501780801913298 bla bla another number: 1256 eee"
print(sentence)
for c in string.punctuation:
        sentence = sentence.replace(c, " ")

print(sentence)
sentence = re.sub(r"\b\d+\b","", sentence)

print(sentence)
#remove multiple spaces
sentence = re.sub(r" +"," ",sentence)


# print(sentence)

In [0]:
spark_df_clean = spark_df.select("documents").rdd.map(remove_punctuation).toDF(schema=["documents","documents_clean"])
spark_df_clean.show()

In [0]:
from  pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol="documents_clean",outputCol="words")

df_words = tokenizer.transform(spark_df_clean)
#check
df_words.show(5)

In [0]:
from pyspark.sql.functions import array_size

df_words.select(array_size("words")).show(5)